## 檢查 GPU

In [0]:
! nvidia-smi

Tue May 26 06:15:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 實作推薦演算法 (implicit)

In [0]:
! pip install implicit

     |████████████████████████████████| 1.1MB 4.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420311 sha256=88e82974847573058f518e464c5d61ff2ea721e8c4d1de4c3a2e3a6142fe042b
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


### Movielens
- https://grouplens.org/datasets/movielens/

In [0]:
from implicit.datasets.movielens import get_movielens

In [0]:
titles, ratings = get_movielens('100k')

In [0]:
len(titles)

1683

In [0]:
ratings

<1683x944 sparse matrix of type '<class 'numpy.float32'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [0]:
import implicit
model = implicit.als.AlternatingLeastSquares(factors=64)

In [0]:
model.fit(ratings)

In [0]:
user_items = ratings.T.tocsr()

In [31]:
user_items.shape

(944, 1683)

In [0]:
recommendations = model.recommend(1, user_items)

In [0]:
for movieid, score in recommendations:
  print(titles[movieid], score)

Batman (1989) 1.1476026
Schindler's List (1993) 1.1209066
Dave (1993) 1.116436
Trainspotting (1996) 1.0858946
E.T. the Extra-Terrestrial (1982) 1.0422022
Piano, The (1993) 1.0099155
It's a Wonderful Life (1946) 0.9585745
One Flew Over the Cuckoo's Nest (1975) 0.94594467
Fantasia (1940) 0.9283118
Clueless (1995) 0.91164553


In [0]:
recommendations = model.recommend(10, user_items)

In [0]:
for movieid, score in recommendations:
  print(titles[movieid], score)

Godfather: Part II, The (1974) 1.1627693
Babe (1995) 1.135076
Annie Hall (1977) 1.0498819
Boot, Das (1981) 0.9982157
People vs. Larry Flynt, The (1996) 0.98383594
Ed Wood (1994) 0.98232365
Scream (1996) 0.96729803
This Is Spinal Tap (1984) 0.88957113
Professional, The (1994) 0.8842394
Grifters, The (1990) 0.8819241


In [0]:
for idx, e in enumerate(titles):
  if 'Star' in e:
    print(titles[idx], idx)

Star Wars (1977) 50
Stargate (1994) 62
Lone Star (1996) 124
Unhook the Stars (1996) 146
Star Trek: First Contact (1996) 222
Star Trek VI: The Undiscovered Country (1991) 227
Star Trek: The Wrath of Khan (1982) 228
Star Trek III: The Search for Spock (1984) 229
Star Trek IV: The Voyage Home (1986) 230
Starship Troopers (1997) 271
Star Trek: Generations (1994) 380
Star Trek: The Motion Picture (1979) 449
Star Trek V: The Final Frontier (1989) 450
Evening Star, The (1996) 1061
Star Maker, The (Uomo delle stelle, L') (1995) 1068
Star Maps (1997) 1265
Star Kid (1997) 1293
Stars Fell on Henrietta, The (1995) 1464
Frankie Starlight (1995) 1545


In [0]:
related = model.similar_items(222)

In [0]:
for movieid, score in related:
  print(titles[movieid], score)

Star Trek: First Contact (1996) 0.17811833
Star Trek: The Wrath of Khan (1982) 0.11140552
Star Trek: Generations (1994) 0.09613714
Star Trek VI: The Undiscovered Country (1991) 0.09307335
Star Trek III: The Search for Spock (1984) 0.09121225
Independence Day (ID4) (1996) 0.08745548
Return of the Jedi (1983) 0.08634575
Star Trek: The Motion Picture (1979) 0.08504435
Toy Story (1995) 0.08314392
Mystery Science Theater 3000: The Movie (1996) 0.083064474


In [38]:
model.similar_users(1)

[(1, 25.549122),
 (715, 16.466887),
 (738, 16.142159),
 (916, 15.717207),
 (886, 15.533796),
 (44, 15.45177),
 (268, 15.022598),
 (806, 14.729637),
 (823, 14.728788),
 (64, 14.525263)]

In [0]:
import numpy as np
np_titles = np.array(titles)
user1 = set(np_titles[user_items[1].toarray()[0] > 0].tolist())
user715 = set(np_titles[user_items[715].toarray()[0] > 0].tolist())

In [0]:
#user1 & user715

In [0]:
#userid, productid, rating
#使用者1, A保險, 1
#使用者1, B保險, 1
#使用者1, C保險, 1
#使用者2, B保險, 1

## 安裝 Spark
- https://spark.apache.org/downloads.html

In [0]:
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
! wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
! tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
! pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## 推薦系統 (Spark)

### 讀取資料 (方法一)

In [56]:
! wget https://raw.githubusercontent.com/ywchiu/taipeifubon/master/data/u.data

--2020-05-26 07:02:54--  https://raw.githubusercontent.com/ywchiu/taipeifubon/master/data/u.data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1979173 (1.9M) [text/plain]
Saving to: ‘u.data’

u.data              100%[===================>]   1.89M  --.-KB/s    in 0.1s    

2020-05-26 07:02:55 (16.2 MB/s) - ‘u.data’ saved [1979173/1979173]



In [57]:
! head u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


In [0]:
rawData = sc.textFile("u.data")

In [59]:
rawData

u.data MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [60]:
rawData.take(3)

['196\t242\t3\t881250949', '186\t302\t3\t891717742', '22\t377\t1\t878887116']

In [0]:
rawRatings = rawData.map(lambda e: e.split())

In [62]:
rawRatings.take(3)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116']]

In [0]:
from pyspark.sql import Row

In [0]:
ratingsRDD  = rawRatings.map(\
               lambda p: Row(userId=int(p[0]),   \
                             movieId=int(p[1]),  \
                             rating=float(p[2]), \
                             timestamp=int(p[3])))


In [65]:
ratingsRDD.take(3)

[Row(movieId=242, rating=3.0, timestamp=881250949, userId=196),
 Row(movieId=302, rating=3.0, timestamp=891717742, userId=186),
 Row(movieId=377, rating=1.0, timestamp=878887116, userId=22)]

In [0]:
ratings = spark.createDataFrame(ratingsRDD)

In [67]:
ratings.show(3)

+-------+------+---------+------+
|movieId|rating|timestamp|userId|
+-------+------+---------+------+
|    242|   3.0|881250949|   196|
|    302|   3.0|891717742|   186|
|    377|   1.0|878887116|    22|
+-------+------+---------+------+
only showing top 3 rows



### 讀取資料 (方法二)

In [0]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType, FloatType

schema = StructType([
    StructField("userid", IntegerType()),
    StructField("movieid", IntegerType()),
    StructField("rating", FloatType()),
    StructField("timestamp", IntegerType()),
                
])

In [75]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
# hadoop : hdfs://
df = sqlContext.read.csv('u.data', sep ='\t', schema = schema)
df.show(3)

+------+-------+------+---------+
|userid|movieid|rating|timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
+------+-------+------+---------+
only showing top 3 rows



###將資料分為訓練資料集與測試資料集

In [0]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [78]:
training.count()

79859

In [79]:
test.count()

20141

In [0]:
from pyspark.ml.recommendation import ALS
als = ALS(rank=50, maxIter=10, regParam=0.01, \
            userCol="userId", itemCol="movieId", \
            ratingCol="rating")

model = als.fit(training)

In [83]:
model.userFactors.count()

943

In [84]:
model.itemFactors.count()

1651

In [0]:
userRecs = model.recommendForAllUsers(10)

In [0]:
user3 = userRecs.take(3)

In [93]:
for movieid, rating in user3[0].recommendations:
  print(titles[movieid], rating)

Boot, Das (1981) 5.922089576721191
Ben-Hur (1959) 5.366769790649414
Gandhi (1982) 5.3346991539001465
Great Escape, The (1963) 5.01218318939209
Home Alone (1990) 4.985595226287842
Mrs. Doubtfire (1993) 4.967797756195068
Jurassic Park (1993) 4.961256980895996
Babe (1995) 4.957892417907715
Homeward Bound: The Incredible Journey (1993) 4.954180717468262
Aristocats, The (1970) 4.949374198913574


In [0]:
movieRecs = model.recommendForAllItems(10)

In [95]:
movieRecs.take(3)

[Row(movieId=1580, recommendations=[Row(userId=405, rating=1.0197834968566895), Row(userId=731, rating=0.3146740794181824), Row(userId=432, rating=0.31279855966567993), Row(userId=99, rating=0.3033958375453949), Row(userId=608, rating=0.2955706715583801), Row(userId=121, rating=0.2582400143146515), Row(userId=190, rating=0.25719985365867615), Row(userId=563, rating=0.2546844780445099), Row(userId=690, rating=0.2510766386985779), Row(userId=912, rating=0.24690771102905273)]),
 Row(movieId=471, recommendations=[Row(userId=848, rating=5.5001959800720215), Row(userId=654, rating=5.368454456329346), Row(userId=16, rating=5.3555216789245605), Row(userId=907, rating=5.349058628082275), Row(userId=717, rating=5.314947605133057), Row(userId=523, rating=5.287059307098389), Row(userId=296, rating=5.285308837890625), Row(userId=295, rating=5.234066963195801), Row(userId=130, rating=5.230841636657715), Row(userId=798, rating=5.155935764312744)]),
 Row(movieId=1591, recommendations=[Row(userId=440, 